# **Introduction:**

This file serves to host an ANN model created for use in multi-robot task allocation (MRTA). This model performs regression based on the load history of a robot, its distance to the current task, as well as the total distance that it has travelled thus far. The goal for designing this ANN is to comapre its performance against an ANFIS to determine which is better at approximating an FIS. This will be achieved through comparing the coefficient of determination ($R^{2}$), root mean squared error (RMSE), and mean absolute error (MAE).

**Date Created:** 18/12/2024

**Date Modified:** 6/1/2025

# **Import Packages:**

This section imports all necessary pacakges for the ANN implementation:

In [2]:
# import packages:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import json

# **Data Loading:**

This section loads the data that was generated from the FIS. Minimal discovery is performed here, as the bulk of the data discovery was performed within the first grid search.

In [3]:
# get the path to the data CSV:
files_in_dir = os.listdir(os.getcwd())
data_path = os.path.join(os.getcwd(), files_in_dir[files_in_dir.index('V3_Data.csv')])

# load the CSV as a pandas dataframe:
df = pd.read_csv(data_path)
print(f'data successfully loaded')

data successfully loaded


# **Data Pre-Processing:**

This section will split the data into training, validation, and testing, alongside performing some pre-processing.

In [4]:
# get the feature and label dataframes:
x_data = df.drop(['Suitability'], axis = 1)
y_data = df['Suitability']

Need to first standardize the values:

In [5]:
# define a scaler:
scaler = StandardScaler()
x_data = scaler.fit_transform(x_data)

Split data into training, validation, and testing:

In [6]:
# split dataset:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size = 0.5)

# get split results:
print(f"there are {x_train.shape[0]} training examples")
print(f"there are {x_val.shape[0]} validation examples")
print(f"there are {x_test.shape[0]} testing examples")

# get input shape:
INPUT_SHAPE = x_data.shape[1]

there are 12000 training examples
there are 1500 validation examples
there are 1500 testing examples


# **Create Model**

Need to define the relevant metrics and loss functions, and then define a function for creating models:

In [ ]:
LOSS_FUNCTION = 'mse'
METRICS = ['mae', keras.metrics.RootMeanSquaredError(), keras.metrics.R2Score]

Define model generation function:

In [ ]:
def make_model(layers, neurons, rate, norm, drop):
    # instantiate model:
    model = keras.Sequential()

    # add hidden layers:
    for i in range(layers):
        if i == 0:
            model.add(Input(shape = (INPUT_SHAPE, )))
            model.add(Dense(neurons, activation = 'relu', name = f'hidden_layer_{i+1}'))
        else:
            model.add(Dense(neurons, activation = 'relu', name = f'hidden_layer_{i+1}'))

        if norm == True:
            model.add(BatchNormalization(name = f'batch_norm_layer_{i+1}'))

        if drop == True:
            model.add(Dropout(0.2, name = f'dropout_layer_{i+1}'))
    
    # add output layer:
    model.add(Dense(1, activation = 'linear', name = 'output_layer'))

    # compile the model:
    model.compile(optimizer = Adam(learning_rate = rate),
                  loss = LOSS_FUNCTION,
                  metrics = METRICS)
    
    return model 
